# CISC/CMPE 452/COGS 400 Assignment 3 - Unsupervised Learning (20 points)  

Please put your name and student id

    Vincent Hung, #20273117

- Make sure to run all the cells from the beginning before submission. Do not clear out the outputs. You will only get credit for code that has been run.
- You can only use Numpy to build the models. Other packages such as Pandas, Sklearn and Scipy can be used for evaluation metrics calculating, data processing, and file reading and writing.

### Files need to be uploaded for this assignment: A3.ipynb, output.wav, and output.csv

## Part 1 Principle Component Analysis Network (10 points)
The dataset "data/sound.csv" contains two sounds recorded by the two microphones. The goal of this assignment is using PCA network to find the approximation of the first principal component.
- Build a PCA network (refer to Principal Component Analysis slide #22 and #23) to reduce the number of features from 2 to 1 (3 points)  
- Train the model and generate the processed data (1 point)  
- Save the data into output.wav and output.csv files (1 point)  
- Compare the sound_o.wav (audio with noise) and output.wav (audio is denoised)  

In [333]:
import numpy as np
import pandas as pd
from scipy.io import wavfile

In [334]:
samrate = 8000
#Sampel rate

In [335]:
# read csv into array
txtData = np.genfromtxt('data/sound.csv', delimiter=',')
txtData.shape

(50000, 2)

In [336]:
# save array to WAV file
#Turns text data into actual sound file
scaledData = np.int16(txtData * samrate)
wavfile.write('data/sound_o.wav', samrate, scaledData)

In [337]:
# read WAV file into array
# The data in sound.csv is processed
# If you use the data generated here, you need to process the data by adding wavData = wavData / samrate
#This is just previous block but reversed
samrate, wavData = wavfile.read('data/sound_o.wav')
samrate, wavData.shape

(8000, (50000, 2))

In [338]:
# save array to csv file
np.savetxt('data/sound_o.csv', txtData, delimiter=',')

In [339]:
# build PCA model and only Numpy can be used
class PCA(object):
    def __init__(self, lr, epoch):
        self.lr = lr
        self.epoch = epoch

    def train(self, x, n_components):
        #x is the data
        #n_components is how many components we want to reduce to
        #Normalize data
        x = x - np.mean(x)

        #Cov matrix
        cov_matrix = np.cov(x.T)

        # Eigenvectors
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # Sort eigenvalues and eigenvectors
        sorted_idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, sorted_idx]

        self.components = eigenvectors[:, :n_components]

        # Transform data to lower dimension
        return np.dot(x, self.components)




In [340]:
# initialize and train the model
pca = PCA(0.1, 100)
pca_data = pca.train(txtData, 1)

# Save the denoised audio
output_scaled = np.int16(pca_data * samrate)
wavfile.write('output.wav', samrate, output_scaled)



In [341]:
# save the data
# Save processed data to CSV
np.savetxt('output.csv', pca_data, delimiter=',')

## Part 2 K-Means Clustering Algorithm (10 points)
The dataset is [Palmer Archipelago (Antarctica) penguin data](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data) which has 6 features and 1 label called species (Chinstrap, Adélie, or Gentoo)  
The dataset is saved in the "data/penguins_size.csv" file and preprocessed into x_train, x_test, y_train, y_test  
- Build a K-Means clustering algorithm (refer to Unsupervised Learning slide #29) to cluster the preprocessed data (2 points)  
- Standardize the data and train the model with the training set (1 point)  
- Evaluate the model and print the confusion matrixes with both training and test sets (2 points)  


In [342]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [343]:
# load the dataset
data = pd.read_csv('data/penguins_size.csv')
data.head()

species     island  culmen_length_mm  culmen_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen              39.1             18.7              181.0   
1  Adelie  Torgersen              39.5             17.4              186.0   
2  Adelie  Torgersen              40.3             18.0              195.0   
3  Adelie  Torgersen               NaN              NaN                NaN   
4  Adelie  Torgersen              36.7             19.3              193.0   

   body_mass_g     sex  
0       3750.0    MALE  
1       3800.0  FEMALE  
2       3250.0  FEMALE  
3          NaN     NaN  
4       3450.0  FEMALE

In [344]:
# data preprocessing
data = data.dropna()
data = data[data['sex'] != '.']

cleanup_nums = {"species": {"Adelie": 0, "Chinstrap": 1, "Gentoo": 2},
                "island": {"Biscoe": 0, "Dream": 1, "Torgersen": 2},
                "sex": {"MALE": 0.0, "FEMALE": 1.0}}
data = data.replace(cleanup_nums)

data.head()

C:\Users\Allclan\AppData\Local\Temp\ipykernel_38300\3501139218.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace(cleanup_nums)


species  island  culmen_length_mm  culmen_depth_mm  flipper_length_mm  \
0        0       2              39.1             18.7              181.0   
1        0       2              39.5             17.4              186.0   
2        0       2              40.3             18.0              195.0   
4        0       2              36.7             19.3              193.0   
5        0       2              39.3             20.6              190.0   

   body_mass_g  sex  
0       3750.0  0.0  
1       3800.0  1.0  
2       3250.0  1.0  
4       3450.0  1.0  
5       3650.0  0.0

In [345]:
x = np.array(data.drop(['species'], axis=1).copy())
y = np.array(data['species'].copy()).astype(int)

In [346]:
# data standardization
x = (x - np.mean(x)) / np.std(x)
print(x)

[[-0.46904375 -0.44562486 -0.45850209 -0.35605231  1.89683286 -0.47030623]
 [-0.46904375 -0.44537236 -0.4593227  -0.35289613  1.92839472 -0.46967499]
 [-0.46904375 -0.44486737 -0.45894396 -0.34721499  1.58121432 -0.46967499]
 ...
 [-0.47030623 -0.43849188 -0.4603958  -0.33017159  3.15930704 -0.47030623]
 [-0.47030623 -0.44177431 -0.46096392 -0.33648396  2.81212664 -0.46967499]
 [-0.47030623 -0.4388075  -0.46014331 -0.33585273  2.93837406 -0.47030623]]


In [347]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((266, 6), (67, 6), (266,), (67,))

In [348]:
# calculate the confusion matrix
def evaluator(y, y_pred):    
    confusion_matrix = metrics.confusion_matrix(y, y_pred)
    print('Confusion matrix:\n', confusion_matrix)

In [349]:
# setup a baseline model
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) # n_clusters - the number of clusters
km.fit(x_train)
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)
#   A C G
# A # # #
# C # # #
# G # # #

Confusion matrix:
 [[36  0 71]
 [18  0 40]
 [33 68  0]]
Confusion matrix:
 [[ 8  0 31]
 [ 2  0  8]
 [ 6 12  0]]


In [ ]:
# build K-means model and only Numpy can be used
class KMeans(object):
    def __init__(self, k):
        self.k = k
        self.centroids = None
    
    def train(self, x, y, x_test, y_test, learning_rate, n_iters):
        self.centroids = x[np.random.choice(len(x), self.k)] #pick random 3 points as centroids
        #print(self.centroids)
        
        for n in range(n_iters):
            cluster = self.predict(x)
            new_centroids = np.zeros_like(self.centroids)
            for idx in range(self.k):
                clustergroup = x[cluster == idx]


                new_centroids[idx] = clustergroup.mean(axis=0)
                print(new_centroids[idx])
            if np.allclose(new_centroids, self.centroids):
                print(n)
                break
            else:
                self.centroids = new_centroids


    def predict(self, x):
        cluster_assignment = []
        for point in x:
            dist = np.linalg.norm(point - self.centroids, axis=1)
            closest = np.argmin(dist)
            cluster_assignment.append(closest)
        return np.array(cluster_assignment)


In [351]:
# initialize and train the model
kmeans = KMeans(3)

kmeans.train(x_train, y_train, x_test, y_test, 1, 100)


[-0.47023094 -0.44046956 -0.46050757 -0.33463658  2.72670694 -0.47003983]
[-0.46966156 -0.44345537 -0.45877473 -0.34915795  1.81759246 -0.46997046]
[-0.46971444 -0.44321827 -0.45886111 -0.34433498  2.22330079 -0.47018787]
0


In [352]:
# evaluate the model and print the confusion matrixes for both training and test sets
y_pred = kmeans.predict(x_train)
evaluator(y_train, y_pred)
y_pred = kmeans.predict(x_test)
evaluator(y_test, y_pred)

Confusion matrix:
 [[ 9 87 11]
 [ 4 52  2]
 [96  2  3]]
Confusion matrix:
 [[ 5 33  1]
 [ 1  9  0]
 [16  0  2]]
